In [1]:
pip install pandas scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

In [9]:
# Load your dataset
df = pd.read_csv('../train_data/defender.csv')

# Preview the data to ensure everything looks right
print(df.head())

# Check for any missing values and handle them (if any)
df.fillna(0, inplace=True)  # Simple way to handle missing values

         name      team opponent_team        date  was_home position  minutes  \
0  Nathan Aké  Man City      West Ham  2022-08-07     False       DC       90   
1  Nathan Aké  Man City   Southampton  2022-10-08      True       DC       90   
2  Nathan Aké  Man City     Liverpool  2022-10-16     False       DL       90   
3  Nathan Aké  Man City      Brighton         NaN      True      NaN        0   
4  Nathan Aké  Man City  Leeds United         NaN     False      NaN        0   

   goals    xG  assists  ...  last_season_xG  last_season_xA  \
0    0.0  0.01      0.0  ...             0.0             0.0   
1    0.0  0.00      0.0  ...             0.0             0.0   
2    0.0  0.00      0.0  ...             0.0             0.0   
3    NaN   NaN      NaN  ...             0.0             0.0   
4    NaN   NaN      NaN  ...             0.0             0.0   

   last_season_expected_goals_conceded  last_season_clean_sheet_probability  \
0                                  0.0           

In [10]:
# Define the feature columns you want to use for predictions
features = [
    'minutes','goals','xG','assists','xA','total_points','shots','key_passes','ict_index','bonus','clean_sheets','goals_conceded','expected_goals_conceded','clean_sheet_probability','last_season_expected_goals_conceded','last_season_clean_sheet_probability','starts','form','xG&A_form','minutes_per_game','last_season_xG','last_season_xA','next_week_specific_fixture_difficulty', 'next_week_holistic_fixture_difficulty'
]

# Define the target column (points scored in the next gameweek)
target = 'next_week_points'

# Split the dataset into features (X) and target (y)
X = df[features]
y = df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
rf_predictions = rf_model.predict(X_test)

# Evaluate the model's performance
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_mae = mean_absolute_error(y_test, rf_predictions)

print(f"Random Forest MSE: {rf_mse}")
print(f"Random Forest MAE: {rf_mae}")

Random Forest MSE: 4.1601047780031415
Random Forest MAE: 1.1898032977972048


In [12]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
xgb_predictions = xgb_model.predict(X_test)

# Evaluate the model's performance
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_mae = mean_absolute_error(y_test, xgb_predictions)

print(f"XGBoost MSE: {xgb_mse}")
print(f"XGBoost MAE: {xgb_mae}")

XGBoost MSE: 4.0056239150339215
XGBoost MAE: 1.1492506514676482


In [13]:
# Feature importance for Random Forest
rf_feature_importance = rf_model.feature_importances_
rf_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_feature_importance
}).sort_values(by='importance', ascending=False)

print(rf_importance_df)

# Feature importance for XGBoost
xgb_feature_importance = xgb_model.feature_importances_
xgb_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_feature_importance
}).sort_values(by='importance', ascending=False)

print(xgb_importance_df)

                                  feature  importance
0                                 minutes    0.191468
19                       minutes_per_game    0.120670
22  next_week_specific_fixture_difficulty    0.100810
23  next_week_holistic_fixture_difficulty    0.089418
17                                   form    0.065066
8                               ict_index    0.063263
12                expected_goals_conceded    0.061888
13                clean_sheet_probability    0.056956
18                              xG&A_form    0.043337
5                            total_points    0.025635
4                                      xA    0.023494
21                         last_season_xA    0.022816
2                                      xG    0.022713
14    last_season_expected_goals_conceded    0.020708
11                         goals_conceded    0.019928
20                         last_season_xG    0.019856
15    last_season_clean_sheet_probability    0.018445
6                           